Here do also the t-test for the results of the different models by metric.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from tabulate import tabulate

In [ ]:
def display_structure(dictionary):
    rows = []
    for key, value in dictionary.items():
        if isinstance(value, dict):
            for subkey, subvalue in value.items():
                if isinstance(subvalue, dict):
                    for subsubkey, subsubvalue in subvalue.items():
                        rows.append([key, subkey, subsubkey, type(subsubvalue).__name__])
                        key = ''
                        subkey = ''
                else:
                    rows.append([key, subkey, '', type(subvalue).__name__])
                    key = ''
                    subkey = ''
        else:
            rows.append([key, '', '', type(value).__name__])
    print(tabulate(rows, headers=['Key', 'Subkey', 'Sub-subkey', 'Type']))

In [ ]:
def display_results(dictionary):
    rows = []
    config_rows = []
    f1_rows = []
    r2_rows = []
    pearson_rows = []
    average_rows = []
    for key, value in dictionary.items():
        if key == 'config':
            target_rows = config_rows
        elif key == 'f1':
            target_rows = f1_rows
            
            # aggregate secondary f1 scores
            f1_secondary_means = [v['mean'] for k,v in value.items() if k != 'target']
            f1_secondary_stds = [v['std'] for k,v in value.items() if k != 'target']
            
        elif key == 'r2':
            target_rows = r2_rows
        elif key == 'pearson':
            target_rows = pearson_rows
        elif key == 'average':
            target_rows = average_rows
        else:
            target_rows = rows
        if isinstance(value, dict):
            for subkey, subvalue in value.items():
                if isinstance(subvalue, dict):
                    for subsubkey, subsubvalue in subvalue.items():
                        target_rows.append([key, subkey, subsubkey, format_value(subsubvalue)])
                        key = ''
                        subkey = ''
                else:
                    target_rows.append([key, subkey, '', format_value(subvalue)])
                    key = ''
        else:
            target_rows.append([key, '', '', format_value(value)])
    
    # add f1 secondary scores
    average_rows.append(['', 'secondary f1', 'mean', format_value(np.mean(f1_secondary_means))])
    average_rows.append(['', '', 'std', format_value(np.mean(f1_secondary_stds))])
    print(
        tabulate(
            config_rows + average_rows + f1_rows + r2_rows + pearson_rows, # + rows, 
            headers=['Key', 'Subkey', 'Sub-subkey', 'Value']
        )
    )
    #return average_rows

def format_value(value):
    if isinstance(value,(int,float)):
        return f'{value:.2f}'
    elif isinstance(value,list):
        return f'{len(value)} cls'
    else:
        return value

In [ ]:
# import any json file from the results folders

file = "./results/targCls_2_music_openl3_env_voice_True_TotCls_5_dropNs_True_rep_5_fold_5.json"

with open(file) as f:
    data = json.load(f)
    print(file)
    display_results(data)
    #print(data["config"]["classifications"].keys())
